# Prediction of LCZ classes on the entire study area

In [ ]:
#making patches of 10 by 10 pixel from the large imagery and store them in a tif file
import os
import rasterio
from rasterio.windows import Window

def create_patches(input_file, patch_size, stride, output_dir):
    patch_files = []  # List to store the patch file paths

    with rasterio.open(input_file) as src:
        width = src.width
        height = src.height
        num_bands = src.count

        for i in range(0, height-patch_size+1, stride):
            for j in range(0, width-patch_size+1, stride):
                window = Window(j, i, patch_size, patch_size)
                patch = src.read(window=window)

                # Here you can perform any operations on the patch
                # For example, you can process it or analyze it further

                # Generate a unique filename for the patch
                patch_filename = f"patch_{i}_{j}.tif"

                # Set the path for the patch file
                patch_filepath = os.path.join(output_dir, patch_filename)

                # Set the parameters for the TIFF file
                profile = src.profile.copy()
                profile.update(width=patch_size, height=patch_size, transform=rasterio.windows.transform(window, src.transform))

                # Write the patch to a new TIFF file
                with rasterio.open(patch_filepath, 'w', **profile) as dst:
                    dst.write(patch)

                # Store the patch file path in the patch_files list
                patch_files.append(patch_filepath)

    return patch_files

# Example usage
input_file = 'Amsterdam_Sen2.tif'
patch_size = 10
stride = 10
output_dir = 'Amsterdam_Sen2_all_patches10by10'
patch_file_list = create_patches(input_file, patch_size, stride, output_dir)



In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('lcz_model_bestweight2')

# Load the best weights
model.load_weights('best_model_weights2.h5')

2023-05-26 18:25:27.944507: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.


In [2]:
import os
import glob
import rasterio
import numpy as np
from rasterio.transform import Affine
import tensorflow as tf

input_directory = 'Amsterdam_Sen2_all_patches10by10'
output_directory = 'lcz_predicted_patches'
num_classes = 15
num_bands = num_classes + 1  # Including the first band for the predicted class

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Get the file paths of all images in the input directory
image_files = glob.glob(os.path.join(input_directory, '*.tif'))

for image_file in image_files:
    # Create the output file path
    output_file = os.path.join(output_directory, f"output_{os.path.basename(image_file)}")

    with rasterio.open(image_file) as src:
        # Read the metadata
        profile = src.profile
        input_image = src.read()
        input_image = input_image.transpose((1, 2, 0))  # Change the order of dimensions

    # Normalize the input image using the maximum value
    max_value = np.amax(input_image)
    input_image_norm = input_image / max_value
    input_image_norm = np.expand_dims(input_image, axis=0)

    # Predict the class probabilities for each pixel
    pred_probs = model.predict(input_image_norm)
    pred_probs = np.round(pred_probs, decimals=2)

    # Get the predicted class for each pixel
    pred_classes = np.argmax(pred_probs, axis=-1)

    # Create the output profile
    output_profile = profile.copy()
    output_profile.update(count=num_bands, dtype='float32')
    output_profile.transform = Affine.identity()

    # Create the output data array
    output_data = np.zeros((num_bands, input_image.shape[0], input_image.shape[1]), dtype=np.float32)
    output_data[0] = pred_classes

    # Reshape and repeat pred_probs to match the shape of output_data
    reshaped_pred_probs = pred_probs.reshape((num_classes, 1, 1))
    repeated_pred_probs = np.repeat(reshaped_pred_probs, input_image.shape[0], axis=1)
    repeated_pred_probs = np.repeat(repeated_pred_probs, input_image.shape[1], axis=2)

    # Assign the repeated pred_probs to the remaining bands of output_data
    output_data[1:] = repeated_pred_probs

    # Write the output data to the TIFF file
    with rasterio.open(output_file, 'w', **output_profile) as dst:
        dst.write(output_data)

1/1 [==============================] - 0s 27ms/step


In [1]:
#merge tif files stored in a folder into mosaic as a single tif
import os
import rasterio
from rasterio.merge import merge

# Folder path containing the TIFF files
folder_path = "lcz_predicted_patches"

# List files in the folder with a .tif extension
files_tif = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".tif")]

output = 'lcz_predicted_mosaic.tif'

# Open the first TIFF file to get the metadata
with rasterio.open(files_tif[0]) as src:
    meta = src.meta.copy()

# Merge the TIFF files
arr, out_trans = merge(files_tif)

# Update the metadata with the merged array shape and affine transform
meta.update({
    "driver": "GTiff",
    "height": arr.shape[1],
    "width": arr.shape[2],
    "transform": out_trans
})

# Write the mosaic raster to disk
with rasterio.open(output, "w", **meta) as dest:
    dest.write(arr)